In [ ]:
#Scraping indeed website

In [1]:
import urllib
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
import urllib.request
import numpy as np

In [ ]:
####################################################################

In [2]:
link = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 1000 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.
i = 0
results = []
df1 = pd.DataFrame(columns=["Title","Location","Company","Salary", "Summary", "Description", "Review"])
for city in set(['Los+Angeles','New+York', 'Jersey+City']):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        url = link.format(city, start)
        # Append to the full set of results
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = None
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:
                location = None
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = None
            try:
                salary = each.find('span', {'class':'no-wrap'}).text
            except:
                salary = None
            try:
                summary = each.find('span', {'class':'summary'}).text.replace('\n', '')
            except:
                summary = None
            try: 
                description_link = ("%s%s" % ("https://www.indeed.com",each.find('a').get('href')))
            except:
                description_link = 'None'
            try:
                review_link = ("%s%s" % ("https://www.indeed.com",each.find('div').find('a').get('href')))            
            except: 
                review_link = 'None'
            df1 = df1.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Summary':summary,\
                                      'Description':description_link, 'job_description': None, 'Review': review_link,\
                           'comp_rating_overall': None, 'wl_bal_rating': None, 'benefit_rating': None, 'jsecurity_rating': None,\
                            'mgmt_rating': None, 'culture_rating': None}, ignore_index=True)
    sleep(1)

In [6]:
df = pd.read_csv('jobs_desc_ratings.csv')

(39799, 14)

In [8]:
df1.shape

(4264, 14)

In [9]:
df2 = pd.concat([df,df1], axis=0)

In [11]:
df2.to_csv('first_jobs_scraping.csv', index = False)

In [12]:
df2 = pd.read_csv('first_jobs_scraping.csv')

In [15]:
df3 = df1.iloc[:2000,:]
df4 = df1.iloc[2000:4265,:]

In [18]:
def get_jobdesc(df):
    for i in range(0,len(df)):
        company_name = df.iloc[i]['Company']
        desc = df.iloc[i]['Description']
        html = requests.get(desc)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        try:
            df.loc[df['Company'] == company_name, 'job_description'] = soup.find(class_="jobsearch-JobComponent-description icl-u-xs-mt--md").text.replace('\n', ' ')
        except:
            df.loc[df['Company'] == company_name, 'job_description'] = 'None'
        sleep(0.10)
    return df

In [19]:
def get_jobdesc_again(df):
    for i in range(0,len(df)):
        if(df.iloc[i]['job_description'] == 'None'):
            company_name = df.iloc[i]['Company']
            desc = df.iloc[i]['Description']
            html = requests.get(desc)
            soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
            try:
                df.loc[df['Company'] == company_name, 'job_description'] = soup.find(class_="jobsearch-JobComponent-description icl-u-xs-mt--md").text.replace('\n', ' ')
            except:
                df.loc[df['Company'] == company_name, 'job_description'] = 'None'
            sleep(0.10)
    return df

In [20]:
def get_reviews(df):
    for i in range(0,len(df)):
        if(df.iloc[i]['Review'] != 'None'):
            company_name = df.iloc[i]['Company']
            cmp_page = df.iloc[i]['Review']
            html = requests.get(cmp_page)
            soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
            try:
                df.loc[df['Company'] == company_name, 'comp_rating_overall'] = float(soup.find(class_="cmp-header-rating-average").text.replace('\n', ' '))
            except:
                df.loc[df['Company'] == company_name, 'comp_rating_overall'] = 'None'
            try:
                df.loc[df['Company'] == company_name, 'wl_bal_rating'] = float(soup.find("div",{"class": "cmp-ReviewCategory cmp-ReviewCategory--textmd"}).\
                        find_all("span", {"class": "cmp-ReviewCategory-rating"})[0].text.replace('\n', ' '))
            except:
                df.loc[df['Company'] == company_name, 'wl_bal_rating'] = 'None'
            try:
                df.loc[df['Company'] == company_name, 'benefit_rating'] = float(soup.find("div",{"class": "cmp-ReviewCategory cmp-ReviewCategory--textmd"}).\
                        find_all("span", {"class": "cmp-ReviewCategory-rating"})[1].text.replace('\n', ' '))
            except:
                df.loc[df['Company'] == company_name, 'benefit_rating'] = 'None'
            try:
                df.loc[df['Company'] == company_name, 'jsecurity_rating'] = float(soup.find("div",{"class": "cmp-ReviewCategory cmp-ReviewCategory--textmd"}).\
                        find_all("span", {"class": "cmp-ReviewCategory-rating"})[2].text.replace('\n', ' '))
            except:
                df.loc[df['Company'] == company_name, 'jsecurity_rating'] = 'None'
            try:
                df.loc[df['Company'] == company_name, 'mgmt_rating'] = float(soup.find("div",{"class": "cmp-ReviewCategory cmp-ReviewCategory--textmd"}).\
                        find_all("span", {"class": "cmp-ReviewCategory-rating"})[3].text.replace('\n', ' '))
            except:
                df.loc[df['Company'] == company_name, 'mgmt_rating'] = 'None'
            try:
                df.loc[df['Company'] == company_name, 'culture_rating'] = float(soup.find("div",{"class": "cmp-ReviewCategory cmp-ReviewCategory--textmd"}).\
                        find_all("span", {"class": "cmp-ReviewCategory-rating"})[4].text.replace('\n', ' '))
            except:
                df.loc[df['Company'] == company_name, 'culture_rating'] = 'None'
            sleep(0.10)
    return df

In [ ]:
df1.to_csv('jobs_desc_ratings.csv', index = False)
df1 =  pd.read_csv('jobs_desc_ratings.csv')

In [25]:
df1_0 = df4.copy() 

In [26]:
df1_0_0 = get_jobdesc(df1_0)
df1_0_1 = get_jobdesc_again(df1_0_0)
df1_0_2 = get_reviews(df1_0_1)

In [ ]:
#df1_0_3.to_csv('job_desc_ratings.csv', index = False)

In [27]:
df4_new = df1_0_2.copy()

In [28]:
df5 = pd.concat([df3_new, df4_new], axis = 0)

In [30]:
df5.to_csv('39800_44064.csv', index = False)

In [69]:
df5 = pd.read_csv('39800_44064.csv')

In [70]:
df5

,Title,Location,Company,Salary,Summary,Description,Review,benefit_rating,comp_rating_overall,culture_rating,job_description,jsecurity_rating,mgmt_rating,wl_bal_rating
0,Python Data Engineer,NaN,SRS Investment Management,NaN,You’ll be joining ...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,NaN,NaN,NaN,What’s the Job?SRS Investment Management is a ...,NaN,NaN,NaN
1,Applied Machine Learning Scientist/Engineer - ...,NaN,Bloomberg,NaN,Information and da...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,https://www.indeed.com/cmp/Bloomberg,3.8,3.9,3.8,"At Bloomberg, our products are fueled by power...",3.4,3.3,3.7
2,EpicCare Analyst,NaN,NYU Langone Health,NaN,Analyze data conve...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,https://www.indeed.com/cmp/Nyu-Langone-Health,4.1,4.1,3.8,"NYU Langone Health is a world-class, patient-c...",3.7,3.6,3.9
3,Data Scientist,"Jersey City, NJ",Carter McKenzie Inc.,"\r\n $120,000 - $150,000 a year",Data engineering experience with large and/or ...,https://www.indeed.com/company/Carter-McKenzie...,None,NaN,NaN,NaN,Responsibilities: Develop new solutions and st...,NaN,NaN,NaN
4,Associate Data Scientist,"Hoboken, NJ",Wiley,NaN,Associate Data Scientist. The Asso...,https://www.indeed.com/rc/clk?jk=361c44860efbc...,https://www.indeed.com/cmp/Wiley,3.7,3.9,3.7,"VP, Data Analytics & Insights This position w...",3.1,3.3,4.1
5,Data Scientist,"New York, NY",Rent the Runway,NaN,Mine the data ecosystem & find fru...,https://www.indeed.com/rc/clk?jk=171566291d439...,https://www.indeed.com/cmp/Rent-the-Runway,3.0,3.5,3.5,Desired: JavaRPythonAbout Rent the Runway: Rec...,3.1,3.1,3.2
6,Data Scientist (Entry-Level),"New York, NY",First Derivatives plc,NaN,First Derivatives plc is currently...,https://www.indeed.com/company/First-Derivativ...,https://www.indeed.com/cmp/First-Derivatives-PLC,3.0,3.2,2.6,Job DescriptionFirst Derivatives plc is curren...,2.9,2.4,2.9
7,Junior Data Scientist,"New York, NY 10155 (Midtown area)",HOMER,NaN,"As a Junior Data Scientist, you wi...",https://www.indeed.com/rc/clk?jk=dbab22c906a9a...,None,NaN,NaN,NaN,HOMER is looking for an exceptional Junior Dat...,NaN,NaN,NaN
8,NLP and Machine Learning Internship,"New York, NY",Mei Messaging,\r\n $12.00 - $12.50 an hour,We combine data and machine learni...,https://www.indeed.com/company/Mei/jobs/Nlp-Ma...,None,NaN,NaN,NaN,Mei is a unique messaging app (www.textmei.com...,NaN,NaN,NaN
9,Data Scientist,"New York, NY",Scienaptic Systems Inc,NaN,Results attribution to core loss e...,https://www.indeed.com/company/Scienaptic-Syst...,None,NaN,NaN,NaN,Job SummaryRole Summary/PurposeThis role is wi...,NaN,NaN,NaN


In [76]:
def get_city_state(df,col):
    df = df.join(df[col].str.split(',', 1, expand=True).rename(columns={0:'City', 1:'State'}))
    df['City'] = df['City'].astype(str)
    df['State'] = df['State'].astype(str)
    df['State'] = [x[0:3] for x in df['State'] if x != None]
    df.drop(['Location'], axis = 1, inplace = True)
    return df

In [77]:
df5 = get_city_state(df5, 'Location')

In [78]:
df5

,Title,Company,Salary,Summary,Description,Review,benefit_rating,comp_rating_overall,culture_rating,job_description,jsecurity_rating,mgmt_rating,wl_bal_rating,City,State
0,Python Data Engineer,SRS Investment Management,NaN,You’ll be joining ...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,NaN,NaN,NaN,What’s the Job?SRS Investment Management is a ...,NaN,NaN,NaN,nan,nan
1,Applied Machine Learning Scientist/Engineer - ...,Bloomberg,NaN,Information and da...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,https://www.indeed.com/cmp/Bloomberg,3.8,3.9,3.8,"At Bloomberg, our products are fueled by power...",3.4,3.3,3.7,nan,nan
2,EpicCare Analyst,NYU Langone Health,NaN,Analyze data conve...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,https://www.indeed.com/cmp/Nyu-Langone-Health,4.1,4.1,3.8,"NYU Langone Health is a world-class, patient-c...",3.7,3.6,3.9,nan,nan
3,Data Scientist,Carter McKenzie Inc.,"\r\n $120,000 - $150,000 a year",Data engineering experience with large and/or ...,https://www.indeed.com/company/Carter-McKenzie...,None,NaN,NaN,NaN,Responsibilities: Develop new solutions and st...,NaN,NaN,NaN,Jersey City,NJ
4,Associate Data Scientist,Wiley,NaN,Associate Data Scientist. The Asso...,https://www.indeed.com/rc/clk?jk=361c44860efbc...,https://www.indeed.com/cmp/Wiley,3.7,3.9,3.7,"VP, Data Analytics & Insights This position w...",3.1,3.3,4.1,Hoboken,NJ
5,Data Scientist,Rent the Runway,NaN,Mine the data ecosystem & find fru...,https://www.indeed.com/rc/clk?jk=171566291d439...,https://www.indeed.com/cmp/Rent-the-Runway,3.0,3.5,3.5,Desired: JavaRPythonAbout Rent the Runway: Rec...,3.1,3.1,3.2,New York,NY
6,Data Scientist (Entry-Level),First Derivatives plc,NaN,First Derivatives plc is currently...,https://www.indeed.com/company/First-Derivativ...,https://www.indeed.com/cmp/First-Derivatives-PLC,3.0,3.2,2.6,Job DescriptionFirst Derivatives plc is curren...,2.9,2.4,2.9,New York,NY
7,Junior Data Scientist,HOMER,NaN,"As a Junior Data Scientist, you wi...",https://www.indeed.com/rc/clk?jk=dbab22c906a9a...,None,NaN,NaN,NaN,HOMER is looking for an exceptional Junior Dat...,NaN,NaN,NaN,New York,NY
8,NLP and Machine Learning Internship,Mei Messaging,\r\n $12.00 - $12.50 an hour,We combine data and machine learni...,https://www.indeed.com/company/Mei/jobs/Nlp-Ma...,None,NaN,NaN,NaN,Mei is a unique messaging app (www.textmei.com...,NaN,NaN,NaN,New York,NY
9,Data Scientist,Scienaptic Systems Inc,NaN,Results attribution to core loss e...,https://www.indeed.com/company/Scienaptic-Syst...,None,NaN,NaN,NaN,Job SummaryRole Summary/PurposeThis role is wi...,NaN,NaN,NaN,New York,NY


In [38]:
df51 = df5[['City', 'State']]

In [66]:
df51  = pd.DataFrame(df51['State'].dropna())

KeyError: 'State'

In [65]:
df51

3                              NJ
4                              NJ
5                              NY
6                              NY
7         NY 10155 (Midtown area)
8                              NY
9                              NY
10       NY 10016 (Gramercy area)
11                             NJ
12                             NY
18                             NJ
19                             NY
20                             NY
21        NY 10018 (Clinton area)
22        NY 10001 (Chelsea area)
23                             NY
24                             NJ
25        NY 10011 (Chelsea area)
26                             NJ
27                             NJ
32                             NY
33                             NY
34                             NY
35                             NY
36                             NY
37                             NY
38                             NY
39                             NY
40        NY 10013 (Tribeca area)
41            

In [59]:
df51['City_len'] = [len(x) for x in df51['State'] if x!= None]

ValueError: Length of values does not match length of index

In [51]:
df51['State_len'] = df51['State'].apply(lambda x: len(x) for x in df51['State'] if x!= None)

TypeError: 'generator' object is not callable

In [24]:
df3_new

,Title,Location,Company,Salary,Summary,Description,Review,benefit_rating,comp_rating_overall,culture_rating,job_description,jsecurity_rating,mgmt_rating,wl_bal_rating
0,Python Data Engineer,None,SRS Investment Management,None,You’ll be joining ...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,None,None,None,None,What’s the Job?SRS Investment Management is a ...,None,None,None
1,Applied Machine Learning Scientist/Engineer - ...,None,Bloomberg,None,Information and da...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,https://www.indeed.com/cmp/Bloomberg,3.8,3.9,3.8,"At Bloomberg, our products are fueled by power...",3.4,3.3,3.7
2,EpicCare Analyst,None,NYU Langone Health,None,Analyze data conve...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,https://www.indeed.com/cmp/Nyu-Langone-Health,4.1,4.1,3.8,"NYU Langone Health is a world-class, patient-c...",3.7,3.6,3.9
3,Data Scientist,"Jersey City, NJ",Carter McKenzie Inc.,"\n $120,000 - $150,000 a year",Data engineering experience with large and/or ...,https://www.indeed.com/company/Carter-McKenzie...,None,None,None,None,Responsibilities: Develop new solutions and st...,None,None,None
4,Associate Data Scientist,"Hoboken, NJ",Wiley,None,Associate Data Scientist. The Asso...,https://www.indeed.com/rc/clk?jk=361c44860efbc...,https://www.indeed.com/cmp/Wiley,3.7,3.9,3.7,"VP, Data Analytics & Insights This position w...",3.1,3.3,4.1
5,Data Scientist,"New York, NY",Rent the Runway,None,Mine the data ecosystem & find fru...,https://www.indeed.com/rc/clk?jk=171566291d439...,https://www.indeed.com/cmp/Rent-the-Runway,3,3.5,3.5,Desired: JavaRPythonAbout Rent the Runway: Rec...,3.1,3.1,3.2
6,Data Scientist (Entry-Level),"New York, NY",First Derivatives plc,None,First Derivatives plc is currently...,https://www.indeed.com/company/First-Derivativ...,https://www.indeed.com/cmp/First-Derivatives-PLC,3,3.2,2.6,Job DescriptionFirst Derivatives plc is curren...,2.9,2.4,2.9
7,Junior Data Scientist,"New York, NY 10155 (Midtown area)",HOMER,None,"As a Junior Data Scientist, you wi...",https://www.indeed.com/rc/clk?jk=dbab22c906a9a...,None,None,None,None,HOMER is looking for an exceptional Junior Dat...,None,None,None
8,NLP and Machine Learning Internship,"New York, NY",Mei Messaging,\n $12.00 - $12.50 an hour,We combine data and machine learni...,https://www.indeed.com/company/Mei/jobs/Nlp-Ma...,None,None,None,None,Mei is a unique messaging app (www.textmei.com...,None,None,None
9,Data Scientist,"New York, NY",Scienaptic Systems Inc,None,Results attribution to core loss e...,https://www.indeed.com/company/Scienaptic-Syst...,None,None,None,None,Job SummaryRole Summary/PurposeThis role is wi...,None,None,None


In [79]:
link = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.
i = 0
results = []
df1 = pd.DataFrame(columns=["Location, alternate"])
for city in set(['Atlanta',
 'Austin',
 'Baltimore',
 'Boston',
 'Charlotte',
 'Chicago',
 'Cincinnati',
 'Columbus',
 'Dallas',
 'Denver',
 'Houston',
 'Indianapolis',
 'Jacksonville',
 'Kansas+City',
 'Los+Angeles',
 'New+York',
 'Miami',
 'Minneapolis',
 'Nashville',
 'Oakland',
 'Philadelphia',
 'Phoenix',
 'Pittsburgh',
 'Portland',
 'San+Diego',
 'San+Francisco',
 'San+Jose',
 'Seattle',
 'St.+Louis',
 'Tomp',
 'Washington%2C+DC']):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        url = link.format(city, start)
        # Append to the full set of results
        html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        for each in soup.find_all(class_= "result" ):
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:
                location = None
            
            df1 = df1.append({'alternate':city, 'Location':location}, ignore_index=True)
    sleep(0.5)

In [82]:
df2 = df1[df1['Location'].notnull()]

,"Location, alternate",Location,alternate
3,NaN,"Cincinnati, OH 45202 (Mount Adams area)",Cincinnati
4,NaN,"Cincinnati, OH 45240",Cincinnati
5,NaN,"Cincinnati, OH",Cincinnati
6,NaN,"Cincinnati, OH",Cincinnati
7,NaN,"Cincinnati, OH",Cincinnati
8,NaN,"Cincinnati, OH",Cincinnati
9,NaN,"Cincinnati, OH",Cincinnati
10,NaN,"Cincinnati, OH",Cincinnati
11,NaN,"Cincinnati, OH",Cincinnati
12,NaN,"Cincinnati, OH",Cincinnati


In [ ]:
def get_city_state(df,col):
    df = df.join(df[col].str.split(',', 1, expand=True).rename(columns={0:'City', 1:'State'}))
    df['City'] = df['City'].astype(str)
    df['State'] = df['State'].astype(str)
    df['State'] = [x[0:3] for x in df['State'] if x != None]
    df.drop(['Location', 'State'], axis = 1, inplace = True)
    return df

In [84]:
df3 = get_city_state(df2, 'Location')

In [87]:
df4 = df3[['alternate', 'City']]

In [91]:
df4

,alternate,City
3,Cincinnati,Cincinnati
4,Cincinnati,Cincinnati
5,Cincinnati,Cincinnati
6,Cincinnati,Cincinnati
7,Cincinnati,Cincinnati
8,Cincinnati,Cincinnati
9,Cincinnati,Cincinnati
10,Cincinnati,Cincinnati
11,Cincinnati,Cincinnati
12,Cincinnati,Cincinnati


In [90]:
df4['alternate'] = df4.alternate.str.replace("+", " ")

C:\Anaconda\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [100]:
df4

,alternate,City
3,Cincinnati,Cincinnati
4,Cincinnati,Cincinnati
5,Cincinnati,Cincinnati
6,Cincinnati,Cincinnati
7,Cincinnati,Cincinnati
8,Cincinnati,Cincinnati
9,Cincinnati,Cincinnati
10,Cincinnati,Cincinnati
11,Cincinnati,Cincinnati
12,Cincinnati,Cincinnati


In [92]:
df4['alternate'].value_counts()

Kansas City         100
Portland            100
New York            100
Phoenix             100
Philadelphia        100
Los Angeles         100
Pittsburgh          100
Columbus            100
Austin              100
San Francisco       100
Boston              100
Houston             100
Baltimore           100
San Diego           100
San Jose            100
Charlotte           100
Denver              100
Indianapolis        100
Minneapolis         100
Washington%2C DC    100
Oakland             100
Seattle             100
Chicago             100
Dallas              100
Atlanta             100
St. Louis           100
Cincinnati           98
Miami                98
Nashville            96
Jacksonville         91
Name: alternate, dtype: int64

In [114]:
df4['City'].value_counts()

San Francisco             157
San Diego                  96
Houston                    94
Austin                     94
Charlotte                  92
New York                   88
Indianapolis               86
St. Louis                  85
Chicago                    83
Cincinnati                 81
Pittsburgh                 80
Jacksonville               77
Atlanta                    76
Columbus                   76
Nashville                  75
Seattle                    71
Phoenix                    66
Portland                   61
Kansas City                60
Boston                     55
Philadelphia               53
Los Angeles                50
Miami                      45
Dallas                     38
Irving                     37
Washington                 37
Denver                     34
Baltimore                  29
Minneapolis                24
Boulder                    24
                         ... 
Rock Hill                   1
Whittier                    1
Edina     

In [160]:
new_df = pd.DataFrame(df4.groupby('City')['alternate'].apply(set)).reset_index()

In [161]:
new_df.columns = ['City', 'center']

In [167]:
new_df['centre'] = new_df['center'].apply(lambda x: list(x))

In [172]:
new_df['centre1'] = [x[0] for x in new_df['centre']]

In [174]:
new_df = new_df[['City', 'centre1']]

In [181]:
new_df = new_df.replace('Washington%2C DC', 'Washington DC')

In [182]:
new_df.to_csv('city_list.csv', index = False)

In [135]:
df4.loc[df4['City'] == 'Lombard', 'alternate'].values

array(['Chicago'], dtype=object)

In [104]:
city_df = pd.DataFrame(df4['City'].value_counts().index)
city_df.columns = ['cities']
city_df

,cities
0,San Francisco
1,San Diego
2,Houston
3,Austin
4,Charlotte
5,New York
6,Indianapolis
7,St. Louis
8,Chicago
9,Cincinnati


In [106]:
#new = city_df.merge(df4, how = 'left', left_on = 'cities', right_on = 'alternate')

In [125]:
city_df['new_city'] = df4.loc[(df4['City'] == city_df['cities']), 'alternate']

ValueError: Can only compare identically-labeled Series objects

In [113]:
new['alternate'].value_counts()

San Francisco    100
Charlotte        100
New York         100
Phoenix          100
Philadelphia     100
Los Angeles      100
Pittsburgh       100
Columbus         100
Austin           100
Boston           100
Houston          100
San Diego        100
Portland         100
San Jose         100
Denver           100
Indianapolis     100
Minneapolis      100
Baltimore        100
Kansas City      100
Oakland          100
Seattle          100
Chicago          100
Dallas           100
Atlanta          100
St. Louis        100
Cincinnati        98
Miami             98
Nashville         96
Jacksonville      91
Name: alternate, dtype: int64

In [180]:
new

,cities,alternate,City
0,San Francisco,San Francisco,San Francisco
1,San Francisco,San Francisco,San Francisco
2,San Francisco,San Francisco,San Francisco
3,San Francisco,San Francisco,San Francisco
4,San Francisco,San Francisco,San Francisco
5,San Francisco,San Francisco,San Francisco
6,San Francisco,San Francisco,San Francisco
7,San Francisco,San Francisco,San Francisco
8,San Francisco,San Francisco,San Francisco
9,San Francisco,San Francisco,San Francisco
